In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
#     print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
#         print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [7]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [8]:
C = torch.randn((27, 2))


In [9]:
C[5]

tensor([-0.1849, -0.3935])

In [10]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.1849, -0.3935])

In [11]:
C[X].shape

torch.Size([228146, 3, 2])

In [12]:
X[13,2]

tensor(1)

In [13]:
C[X][13,2]

tensor([-0.2427,  0.6420])

In [14]:
C[1]

tensor([-0.2427,  0.6420])

In [15]:
emb = C[X]
emb.shape
emb

tensor([[[ 0.1275, -0.0632],
         [ 0.1275, -0.0632],
         [ 0.1275, -0.0632]],

        [[ 0.1275, -0.0632],
         [ 0.1275, -0.0632],
         [-0.1849, -0.3935]],

        [[ 0.1275, -0.0632],
         [-0.1849, -0.3935],
         [-1.0764,  0.3586]],

        ...,

        [[ 0.6390,  0.0341],
         [ 0.6390,  0.0341],
         [ 0.0294, -0.3270]],

        [[ 0.6390,  0.0341],
         [ 0.0294, -0.3270],
         [ 0.6390,  0.0341]],

        [[ 0.0294, -0.3270],
         [ 0.6390,  0.0341],
         [-0.5241, -1.4327]]])

In [16]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
# emb @ W1 + b1

In [17]:
# F = torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1)

In [18]:
unbinded = torch.unbind(emb, 1)

In [19]:
# Optimize tensor view manipulation

In [20]:
a = torch.arange(18)

In [21]:
# no memory is being changed/copied/created etc. when .view is being called
a.view(3,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [22]:
# always a 1-dimensional vector
a.storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage._TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [23]:
emb.shape

torch.Size([228146, 3, 2])

In [24]:
# more efficient and easier than bind/unbind
emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

RuntimeError: shape '[32, 6]' is invalid for input of size 1368876

In [25]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
# pytorch can infer missing shape of matrix with -1
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h.shape # 100 activations for each of our 32 examples

torch.Size([228146, 100])

In [26]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [27]:
logits = h @ W2 + b2

In [28]:
counts = logits.exp()

In [29]:
prob = counts / counts.sum(1, keepdims=True)

In [30]:
# index into the rows of prob, pluck out the probability assigned to the correct character, and calculate loss
loss = -prob[torch.arange(32), Y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [228146]

In [31]:
# ---- clearer sequence -----

In [84]:
Xtr.shape, Ytr.shape # dataset

(torch.Size([182625, 3]), torch.Size([182625]))

In [85]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2] # cluster parameters into single list

In [86]:
sum(p.nelement() for p in parameters) # number of parameters in total

11897

In [87]:
# emb = C[X] # (32, 3, 2)
# h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
# logits = h @ W2 + b2
# # counts = logits.exp()
# # prob = counts / counts.sum(1, keepdims=True)
# # loss = -prob[torch.arange(32), Y].log().mean()
# # Above 3 commented lines can be done (more efficiently) with the following line
# F.cross_entropy(logits, Y) # how well neural network currently works with this set of parameters

In [88]:
for p in parameters:
    p.requires_grad = True

In [89]:
# step linearly between exponents of possible learning rates and space them exponentially
# creates possible learning rates to use
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [90]:
lri = []
lossi = []
stepi = []

In [95]:
for i in range(200000):
    # minibatch construct of 32 examples
    # better to have approximate gradient and take many more steps than exact gradient with fewer steps 
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # forward pass
    emb = C[Xtr[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
#     print(loss.item())
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    # lr = lrs[i]
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
        
    # track stats
#     lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.log10().item())

print (loss.item())

2.270831823348999


In [96]:
emb = C[Xtr] # (32, 3, 2)
print(emb.shape)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

torch.Size([182625, 3, 10])


tensor(2.0769, grad_fn=<NllLossBackward0>)

In [97]:
emb = C[Xdev] # (32, 3, 2)
print(emb.shape)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

torch.Size([22655, 3, 10])


tensor(2.1277, grad_fn=<NllLossBackward0>)

In [98]:
# training split, dev/validation split, test split
# ~ 80%, 10%, 10%

In [99]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carmah.
ambril.
khyimis.
reity.
skanden.
jazhnevalee.
rhy.
kaeli.
ner.
kentzerian.
kaleigh.
ham.
pris.
quinton.
lilea.
jadis.
wazell.
dearyni.
jaxen.
dus.
